In [1]:
import torch
import torch.nn as nn

### GPU训练

In [2]:
# 检查是否有可用的GPU设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### 定义数据集

自定义输入X为7张高和宽均为2像素的灰度图片  
自定义输出target为 $y_1$=0, $y_2$=1, $y_3$=2



In [3]:
# 定义训练数据和标签
X = torch.tensor([[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2], [5.8, 2.6, 4.0, 1.2], [6.7, 3.0, 5.2, 2.3]], dtype=torch.float32).to(device)
target = torch.tensor([0, 0, 1, 2], dtype=torch.long).to(device)

#### 定义网络层

In [4]:
# 定义神经网络结构
class LinearNet(nn.Module):
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc = nn.Linear(4, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc(x)
        x = self.softmax(x)
        return x


net = LinearNet().to(device)

#### 定义损失函数
` torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean') `
衡量模型输出与真实标签的差异，在分类时相当有用。  
结合了` nn.LogSoftmax() `和` nn.NLLLoss() `两个函数，进行交叉熵计算。  
主要参数：
weight: 各类别的loss设置权值  
ignore_index: 忽略某个类别  
reduction: 计算模式，可为none/sum/mean  
none: 逐个元素计算  
sum: 所有元素求和，返回标量  
mean: 加权平均，返回标量  

In [5]:
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数

#### 定义优化函数
- torch.optim.SGD(params, lr=<required parameter>, momentum=0, dampening=0, weight_decay=0, nesterov=False)
- 构建一个优化器对象optimizer，用来保存当前的状态，并能够根据计算得到的梯度来更新参数，使得模型输出更接近真实标签。
- 学习率（learning rate）控制更新的步伐。
- 主要参数：
    - params: 管理的参数组
    - lr: 初始化学习率
    - momentum: 动量系数
    - weight_decay: L2正则化系数
    - nesterov: 是否采用NAG
- zero_grad(): 清空所管理参数的梯度，因为Pytorch张量梯度不自动清零。
- step(): 执行一步更新

In [6]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1) # 随机梯度下降法


## 开始训练模型

In [8]:
# 训练神经网络
for epoch in range(100000):
    optimizer.zero_grad()
    y_hat = net(X)
    loss = criterion(y_hat, target)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print("Epoch %d, Train loss: %f" % (epoch, loss.item()))

Epoch 0, Train loss: 0.600651
Epoch 100, Train loss: 0.595703
Epoch 200, Train loss: 0.591606
Epoch 300, Train loss: 0.588167
Epoch 400, Train loss: 0.585245
Epoch 500, Train loss: 0.582735
Epoch 600, Train loss: 0.580558
Epoch 700, Train loss: 0.578654
Epoch 800, Train loss: 0.576976
Epoch 900, Train loss: 0.575486
Epoch 1000, Train loss: 0.574156
Epoch 1100, Train loss: 0.572962
Epoch 1200, Train loss: 0.571883
Epoch 1300, Train loss: 0.570905
Epoch 1400, Train loss: 0.570014
Epoch 1500, Train loss: 0.569200
Epoch 1600, Train loss: 0.568452
Epoch 1700, Train loss: 0.567764
Epoch 1800, Train loss: 0.567128
Epoch 1900, Train loss: 0.566539
Epoch 2000, Train loss: 0.565991
Epoch 2100, Train loss: 0.565482
Epoch 2200, Train loss: 0.565006
Epoch 2300, Train loss: 0.564561
Epoch 2400, Train loss: 0.564144
Epoch 2500, Train loss: 0.563752
Epoch 2600, Train loss: 0.563384
Epoch 2700, Train loss: 0.563036
Epoch 2800, Train loss: 0.562708
Epoch 2900, Train loss: 0.562398
Epoch 3000, Train loss